# 제주도 핫플레이스 - 인스타그램 크롤링

In [16]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [17]:
chromedriver = '/Users/hhlee/Desktop/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram에 접속하여 로그인 

In [18]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [19]:
with open('password.txt') as f:
        password = f.read()

In [20]:
email = 'i___n_ji'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [21]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [22]:
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [23]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도맛집 으로 검색

In [24]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [25]:
driver.get(url)
time.sleep(5)

In [26]:
driver.close()